### 根据天气、温度、湿度、风速等特征判断是否举办网球比赛的模拟数据为例，计算信息熵、信息增益

#### ID3 决策树实现原理

In [ ]:
"""
天气	温度	湿度	风速	打网球
晴	    高	    高	    弱	    否
晴	    高	    高	    强	    否
阴	    中	    高	    弱	    是
雨	    低	    中	    弱	    是
雨	    低	    低	    弱	    是
雨	    中	    低	    强	    否
阴	    中	    低	    强	    是
晴	    低	    中	    弱	    否
晴	    中	    中	    弱	    是
雨	    中	    中	    弱	    是
"""

In [7]:
import numpy as np
print("=======================Information Gain信息增益计算========================")
print("（信息增益值越高，说明该特征对于区分不同类别样本的能力越强）")
# 计算信息熵函数
def entropy(p):
    return -np.sum(p * np.log2(p))
#计算基尼指数
def gini(p):
    return 1 - np.sum(p**2)

# 原始数据熵 （最终结果分布情况，6次举办，4次不举办）
p_parent = np.array([0.6, 0.4])
total_entropy=entropy(p_parent)
print(f"原始总熵值: {total_entropy:.3f} bits")  # 输出0.971bits

# 天气特征各子集
p_sunny = np.array([1/3, 2/3]) #3次晴天特征中，1是2否
p_overcast = np.array([2/2]) #2次阴天特征中，2是0否
p_rainy = np.array([4/5, 1/5]) #5次雨天特征中，4是1否
# 条件熵计算
weighted_entropy = (0.3*entropy(p_sunny)) + (0.2*entropy(p_overcast)) + (0.5*entropy(p_rainy))
print(f"基于天气特征的条件熵: {weighted_entropy:.3f} ")  # 输出0.636
# 关于天气特征的信息增益 Gain(天气)=H(D)-H(D|天气)=p_parent-weighted_entropy
hd_weighted_info_gain=total_entropy-weighted_entropy
print(f"天气特征的信息增益: {hd_weighted_info_gain:.3f} ")  # 输出0.334
print(f"温度特征的信息增益: .... ")  
print(f"湿度特征的信息增益: .... ")
print(f"风速特征的信息增益: .... ")
print("=======================Gini coefficient系数计算========================")
print("（基尼系数的值介于 0 和 1 之间，越接近 0 表示数据集的纯度越高，越接近 1 表示数据集的不纯度越高）")
#原始基尼系数
total_gini=gini(p_parent)
print(f"原始总基尼系数: {total_gini:.3f}")  # 输出0.480
#天气特征各子集
p_sunny = np.array([1/3, 2/3]) #3次晴天特征中，1是2否
p_overcast = np.array([2/2]) #2次阴天特征中，2是0否
p_rainy = np.array([4/5, 1/5]) #5次雨天特征中，4是1否
weighted_gini = (0.3*gini(p_sunny)) + (0.2*gini(p_overcast)) + (0.5*gini(p_rainy))
#关于天气特征的基尼增益
ig_weighted_gini=total_gini-weighted_gini
print(f"基于天气特征的基尼系数不纯度: {weighted_gini:.3f} ") 
print(f"天气特征的基尼增益: {ig_weighted_gini:.3f}")  # 输出0.187
print(f"温度特征的基尼系数: .... ")  
print(f"湿度特征的基尼系数: .... ")
print(f"风速特征的基尼系数: .... ")
#同理温度、湿度、风速的信息增益可以计算出
#最终根据不同特征的信息增益大小排序，选择信息增益最大的特征作为根节点，以此类推，直到所有特征都被选择为叶节点。



=======================Information Gain信息增益计算========================
（信息增益值越高，说明该特征对于区分不同类别样本的能力越强）
原始总熵值: 0.971 bits
基于天气特征的条件熵: 0.636 
天气特征的信息增益: 0.334 
温度特征的信息增益: .... 
湿度特征的信息增益: .... 
风速特征的信息增益: .... 
=======================Gini coefficient系数计算========================
（基尼系数的值介于 0 和 1 之间，越接近 0 表示数据集的纯度越高，越接近 1 表示数据集的不纯度越高）
原始总基尼系数: 0.480
基于天气特征的基尼系数不纯度: 0.293 
天气特征的基尼增益: 0.187
温度特征的基尼系数: .... 
湿度特征的基尼系数: .... 
风速特征的基尼系数: .... 


#### C4.5决策树实现原理


In [ ]:
"""
ID3在处理部分场景数据特征时候存在局限性：
1.连续特征处理：ID3只能处理分类特征，无法直接处理温度、湿度等连续值
2.​特征选择偏差：ID3倾向于选择取值多的特征（如温度按1℃分割会有大量分支）
3.​过拟合风险：ID3生成完全拟合训练数据的复杂树结构
4.​缺失值处理：增加对缺失数据的处理能力

天气（分类）：晴/阴/雨
温度（连续）：数值型
湿度（连续）：数值型
风速（分类）：强/弱

天气	温度(℃)	湿度(%)	风速	打网球
晴	    32	    82	    弱	    否
晴	    35	    85	    强	    否
阴	    25	    78	    弱	    是
雨	    18	    65	    弱	    是
雨	    16	    58	    弱	    是
雨	    22	    62	    强	    否
阴	    24	    60	    强	    是
晴	    20	    68	    弱	    否
晴	    26	    72	    弱	    是
雨	    23	    70	    弱	    是
"""

In [ ]:
import pandas as pd
# 数据信息总熵H(D)为 total_entropy 0.971
#对连续值温度排序后分割，排序后温度 = [16,18,20,22,23,24,25,26,32,35]，对应标签  = [是,是,否,否,是,是,是,是,否,否]
"""
候选分割点 = [
    (16+18)/2=17, 
    (18+20)/2=19,
    (20+22)/2=21,
    (22+23)/2=22.5,
    (23+24)/2=23.5,
    (24+25)/2=24.5,
    (25+26)/2=25.5,
    (26+32)/2=29,
    (32+35)/2=33.5
]
以分割点21.5℃为例详细计算，分割点需要合理选择，不一定平分，假如以22.5为分割点最后就会发现温度对最后样本结果影响为0
左子集（≤21℃）: 温度16,18,20 → 标签[是,是,否]
右子集（>21℃）: 温度22,23,24,25,26,32,35 → 标签[否,是,是,是,是,否,否]
"""
#子集信息熵计算
H_left = entropy(np.array([2/3, 1/3]) )#0.918
H_right = entropy(np.array([4/7, 3/7]) )#0.985
#关于温度的条件熵则为
H_condition = 3/10 * H_left + 7/10 * H_right #0.965
#信息增益
IG = total_entropy - H_condition #0.0058
print(IG)
#计算分裂信息，温度对最终结果样本的分布为3/10,7/10
SI = entropy(np.array([3/10, 7/10])) #0.881
print(SI)
#计算增益率，由此可得在此数据集下，温度对样本的增益率较小，通过此方法可以对连续值特征进行处理
Gain_Ratio = IG / SI #0.0065
Gain_Ratio



0.0058021490143456145
0.8812908992306927


np.float64(0.0065836933291952724)